<a href="https://colab.research.google.com/github/DelmiroDaladier/NLP-studies/blob/master/text_summarization/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import re
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [10]:
dataset = pd.read_csv('Reviews.csv', encoding="utf-8")

In [11]:
dataset.sample(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
27732,27733,B0006PXTDY,AD4MP3O3NL291,"C. White ""ccclady""",0.0,0.0,5.0,1.285286e+09,All Gone,There were none left from the Wedding Candy Ba...
21031,21032,B002QWP89S,A1TY4AX8AVRAYQ,Downing,0.0,0.0,5.0,1.350346e+09,Cleaner dog mouth,"This is a good product. My dog likes them, Mak..."
139902,139903,B0061IUKDM,A1U6M8VJHK2ECA,Isabelle Jolly,0.0,0.0,5.0,1.332374e+09,Higgins & Burke Green Tea,I've been drinking tea since childhood. As an...
152385,152386,B000LKZD4W,A1NCBHQ4VIES48,"Loan P. Nguyen ""phuongloan""",0.0,0.0,5.0,1.315699e+09,Taste great,This is the best brand for vegan jerky. I love...
243693,243694,B000SAPXGS,A2BT968XSRPP0K,"CJBouhlal ""Cjb""",14.0,14.0,5.0,1.269043e+09,Excellent Tea.,I grew up drinking authentic Moroccan Tea in M...


In [12]:
dataset.drop_duplicates(subset ='Text', inplace=True)
dataset.dropna(inplace=True)

In [13]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}



In [21]:
stop_words = set(stopwords.words('english'))
def clean_text(text):
    formatted_text = text.lower()
    formatted_text = re.sub(r'\([^)]*\)', '', formatted_text)
    formatted_text = re.sub('"','', formatted_text)
    formatted_text = re.sub(r'<.*?>', '', formatted_text)
    formatted_Text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in formatted_text.split(" ")])    
    formatted_text = re.sub(r"'s\b","", formatted_text)
    formatted_text = re.sub("[^a-zA-Z]", " ", formatted_text) 
    tokens = [word for word in formatted_text.split() if word not in stop_words]
    
    long_words = []
    for token in tokens:
      if len(token) >= 3:
        long_words.append(token)

    return (" ".join(long_words).strip())

    return formatted_text

In [23]:
cleaned_text = []

for text in dataset['Text']:
  cleaned_text.append(clean_text(text))

In [27]:
def summary_cleaner(text):
    formatted_text = re.sub('"','', text)
    formatted_text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in formatted_text.split(" ")])    
    formatted_text = re.sub(r"'s\b","",formatted_text)
    formatted_text = re.sub("[^a-zA-Z]", " ", formatted_text)
    formatted_text = formatted_text.lower()
    tokens=formatted_text.split()
    formatted_text=''
    for i in tokens:
        if len(i)>1:                                 
            formatted_text=formatted_text+i+' '  
    return formatted_text  

In [28]:
cleaned_summaries = []

for summary in dataset['Summary']:
  cleaned_summaries.append(summary_cleaner(summary))


In [32]:
dataset['cleaned_text'] = cleaned_text
dataset['cleaned_summary'] = cleaned_summaries
dataset['cleaned_summary'].replace('', np.nan, inplace=True)
dataset['cleaned_summary'].dropna(axis=0, inplace=True)

In [45]:
dataset['cleaned_summary'].apply(lambda x : '<SOS>' + str(x) + '<EOS>')

0                  <SOS>good quality dog food <EOS>
1                      <SOS>not as advertised <EOS>
2                    <SOS>delight says it all <EOS>
3                         <SOS>cough medicine <EOS>
4                            <SOS>great taffy <EOS>
                            ...                    
273974    <SOS>wonderful but not my favorites <EOS>
273975     <SOS>terrific for morning sickness <EOS>
273976                      <SOS>ginger chews <EOS>
273977        <SOS>love them love the variety <EOS>
273978               <SOS>chimes ginger chews <EOS>
Name: cleaned_summary, Length: 214589, dtype: object

In [48]:
max_len_text=80 
max_len_summary=10